In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/drive')

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import os

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
!rm -R /content/texts

In [ ]:
!wget 'https://github.com/a18091986/MachineLearning/blob/main/Datasets/%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D1%8B%20%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.zip?raw=true' -O /content/text

--2022-05-04 17:23:23--  https://github.com/a18091986/MachineLearning/blob/main/Datasets/%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D1%8B%20%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.zip?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/a18091986/MachineLearning/raw/main/Datasets/%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D1%8B%20%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.zip [following]
--2022-05-04 17:23:23--  https://github.com/a18091986/MachineLearning/raw/main/Datasets/%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D1%8B%20%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/a18091986/MachineLearning/main/Datasets/%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D1%8B%20%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D0

In [ ]:
!unzip -q '/content/text' -d /content/texts/

In [ ]:
def readText(fileName):
  with open(fileName, 'r') as f:
    text = f.read().replace('\n', ' ')
    return text

className = ['Генри', 'Стругацкие', 'Булгаков', 'Клиффорд', 'Макс', 'Брэдберри']
nClasses = len(className)

In [ ]:
trainText = []
testText = []

for i in className:
  for j in os.listdir('/content/texts/Тексты писателей/'):
    if i in j:
      if 'Обучающая' in j:
        trainText.append(readText('/content/texts/Тексты писателей/'+j))
        print(f'{j} добавлен в обучающую выборку')
      if 'Тестовая' in j:
        testText.append(readText('/content/texts/Тексты писателей/'+j))
        print(f'{j} добавлен в тестовую выборку')
  print("*"*100)
      


(О. Генри) Обучающая_50 вместе.txt добавлен в обучающую выборку
(О. Генри) Тестовая_20 вместе.txt добавлен в тестовую выборку
****************************************************************************************************
(Стругацкие) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Стругацкие) Тестовая_2 вместе.txt добавлен в тестовую выборку
****************************************************************************************************
(Булгаков) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Булгаков) Обучающая_5 вместе.txt добавлен в обучающую выборку
****************************************************************************************************
(Клиффорд_Саймак) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Клиффорд_Саймак) Тестовая_2 вместе.txt добавлен в тестовую выборку
****************************************************************************************************
(Макс Фрай) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Макс Фрай) Тест

In [ ]:
cur_time = time.time()
maxWordsCount = 15000
tokenizer = Tokenizer(num_words=maxWordsCount, filters = '')
# tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='unknown', char_level=False)
tokenizer.fit_on_texts(trainText)
items = list(tokenizer.word_index.items())
f'время работы: {round(time.time() - cur_time, 2)}'

'время работы: 3.16'

In [ ]:
trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)

In [ ]:
# Формирование обучающей выборки по листу индексов слов
# (разделение на короткие векторы)
def getSetFromIndexes(wordIndexes, xLen, step): # функция принимает последовательность индексов, размер окна, шаг окна
  xSample = [] # Объявляем переменную для векторов
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen):# Идём по всей длине вектора индексов
    xSample.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаеммся вперёд на step
    
  return xSample

In [ ]:
# Формирование обучающей и проверочной выборки
# Из двух листов индексов от двух классов
def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна

  # Для каждого из 6 классов
  # Создаём обучающую/проверочную выборку из индексов
  nClasses = len(wordIndexes) # Задаем количество классов выборки
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна (например, 6 по 1341*1000)"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step)) # Добавляем в список очередной текст индексов, разбитый на "кол-во окон*длину окна" 

  # Формируем один общий xSamples
  xSamples = [] # Здесь будет список размером "суммарное кол-во окон во всех текстах*длину окна (например, 15779*1000)"
  ySamples = [] # Здесь будет список размером "суммарное кол-во окон во всех текстах*вектор длиной 6"
  
  for t in range(nClasses): # В диапазоне кол-ва классов(6)
    xT = classesXSamples[t] # Берем очередной текст вида "кол-во окон в тексте*длину окна"(например, 1341*1000)
    for i in range(len(xT)): # И каждое его окно
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples) # Переводим в массив numpy для подачи в нейронку
  ySamples = np.array(ySamples) # Переводим в массив numpy для подачи в нейронку

  
  return (xSamples, ySamples) #Функция возвращает выборку и соответствующие векторы классов

In [ ]:
xLen = 5000
step = 100

In [ ]:
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

In [ ]:
xTrain.shape, yTrain.shape, xTest.shape, yTest.shape

((13291, 5000), (13291, 6), (4579, 5000), (4579, 6))

In [ ]:
xTrainBoW = tokenizer.sequences_to_matrix(xTrain.tolist())
xTestBoW = tokenizer.sequences_to_matrix(xTest.tolist())
xTrainBoW.shape, xTestBoW.shape

((13291, 15000), (4579, 15000))

In [ ]:
# Представляем тестовую выборку в удобных для распознавания размерах
def createTestMultiClasses(wordIndexes, xLen, step): #функция принимает последовательность индексов, размер окна, шаг окна

  #Для каждого из 6 классов
  #Создаём тестовую выборку из индексов
  nClasses = len(wordIndexes) #Задаем количество классов
  xTest6Classes01 = []               #Здесь будет список из всех классов, каждый размером "кол-во окон в тексте * 20000 (при maxWordsCount=20000)"
  xTest6Classes = []                 #Здесь будет список массивов, каждый размером "кол-во окон в тексте * длину окна"(6 по 420*1000)
  for wI in wordIndexes:                       #Для каждого тестового текста из последовательности индексов
    sample = (getSetFromIndexes(wI, xLen, step)) #Тестовая выборка размером "кол-во окон*длину окна"(например, 420*1000)
    xTest6Classes.append(sample)              # Добавляем в список
    xTest6Classes01.append(tokenizer.sequences_to_matrix(sample)) #Трансформируется в Bag of Words в виде "кол-во окон в тексте * 20000"
  xTest6Classes01 = np.array(xTest6Classes01)                     #И добавляется к нашему списку, 
  xTest6Classes = np.array(xTest6Classes)                     #И добавляется к нашему списку, 
  
  return xTest6Classes01, xTest6Classes  #функция вернёт тестовые данные: TestBag 6 классов на n*20000 и xTestEm 6 по n*1000

# Распознаём тестовую выборку и выводим результаты
def recognizeMultiClass(model, xTest, modelName):
  print("НЕЙРОНКА: ", modelName)
  print()
  
  totalSumRec = 0 # Сумма всех правильных ответов
  
  #Проходим по всем классам
  for i in range(nClasses):
    #Получаем результаты распознавания класса по блокам слов длины xLen
    currPred = model.predict(xTest[i])
    #Определяем номер распознанного класса для каждохо блока слов длины xLen
    currOut = np.argmax(currPred, axis=1)

    evVal = []
    for j in range(nClasses):
      evVal.append(len(currOut[currOut==j])/len(xTest[i]))

    totalSumRec += len(currOut[currOut==i])
    recognizedClass = np.argmax(evVal) #Определяем, какой класс в итоге за какой был распознан
    
    #Выводим результаты распознавания по текущему классу
    isRecognized = "Это НЕПРАВИЛЬНЫЙ ответ!"
    if (recognizedClass == i):
      isRecognized = "Это ПРАВИЛЬНЫЙ ответ!"
    str1 = 'Класс: ' + className[i] + " " * (11 - len(className[i])) + str(int(100*evVal[i])) + "% сеть отнесла к классу " + className[recognizedClass]
    print(str1, " " * (55-len(str1)), isRecognized, sep='')
  
  #Выводим средний процент распознавания по всем классам вместе
  print()
  sumCount = 0
  for i in range(nClasses):
    sumCount += len(xTest[i])
  print("Средний процент распознавания ", int(100*totalSumRec/sumCount), "%", sep='')

  print()
  
  return totalSumRec/sumCount

In [ ]:
epochs = 10
batch_size = 64
lr = 0.0001

In [ ]:
xTestBoW_, xTestEmbedding = createTestMultiClasses(testWordIndexes, xLen, step)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
model = Sequential()
model.add(Dense(2000, input_dim = maxWordsCount, activation = 'relu'))
model.add(Dropout(0.35))
model.add(Dense(6, activation = 'softmax'))

model.compile(optimizer = Adam(learning_rate = lr), loss = 'categorical_crossentropy', metrics = ['accuracy'])
result = model.fit(xTrainBoW, yTrain, epochs = epochs, batch_size = batch_size, validation_data = (xTestBoW, yTest))

Epoch 1/10
208/208 [==============================] - 7s 22ms/step - loss: 0.0341 - accuracy: 0.9888 - val_loss: 0.2035 - val_accuracy: 0.9616
Epoch 2/10
208/208 [==============================] - 4s 21ms/step - loss: 8.4244e-05 - accuracy: 1.0000 - val_loss: 0.1889 - val_accuracy: 0.9631
Epoch 3/10
208/208 [==============================] - 4s 21ms/step - loss: 4.3250e-05 - accuracy: 1.0000 - val_loss: 0.1778 - val_accuracy: 0.9642
Epoch 4/10
208/208 [==============================] - 4s 20ms/step - loss: 2.5276e-05 - accuracy: 1.0000 - val_loss: 0.1696 - val_accuracy: 0.9640
Epoch 5/10
208/208 [==============================] - 4s 21ms/step - loss: 1.6865e-05 - accuracy: 1.0000 - val_loss: 0.1638 - val_accuracy: 0.9651
Epoch 6/10
208/208 [==============================] - 4s 20ms/step - loss: 1.1824e-05 - accuracy: 1.0000 - val_loss: 0.1580 - val_accuracy: 0.9655
Epoch 7/10
208/208 [==============================] - 4s 20ms/step - loss: 8.4951e-06 - accuracy: 1.0000 - val_loss: 0.154

In [ ]:
model.compile(optimizer = Adam(learning_rate = lr/2), loss = 'categorical_crossentropy', metrics = ['accuracy'])
result = model.fit(xTrainBoW, yTrain, epochs = epochs, batch_size = batch_size, validation_data = (xTestBoW, yTest))

Epoch 1/10
208/208 [==============================] - 5s 22ms/step - loss: 1.4967e-06 - accuracy: 1.0000 - val_loss: 0.1137 - val_accuracy: 0.9764
Epoch 2/10
208/208 [==============================] - 4s 21ms/step - loss: 3.1521e-07 - accuracy: 1.0000 - val_loss: 0.1005 - val_accuracy: 0.9801
Epoch 3/10
208/208 [==============================] - 4s 20ms/step - loss: 1.4941e-07 - accuracy: 1.0000 - val_loss: 0.0963 - val_accuracy: 0.9803
Epoch 4/10
208/208 [==============================] - 4s 20ms/step - loss: 9.0625e-08 - accuracy: 1.0000 - val_loss: 0.0900 - val_accuracy: 0.9825
Epoch 5/10
208/208 [==============================] - 4s 21ms/step - loss: 6.2147e-08 - accuracy: 1.0000 - val_loss: 0.0884 - val_accuracy: 0.9814
Epoch 6/10
208/208 [==============================] - 4s 20ms/step - loss: 4.4030e-08 - accuracy: 1.0000 - val_loss: 0.0853 - val_accuracy: 0.9823
Epoch 7/10
208/208 [==============================] - 4s 21ms/step - loss: 3.4576e-08 - accuracy: 1.0000 - val_loss: 0

In [ ]:
model.compile(optimizer = Adam(learning_rate = lr/4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
result = model.fit(xTrainBoW, yTrain, epochs = epochs, batch_size = batch_size, validation_data = (xTestBoW, yTest))

Epoch 1/10
208/208 [==============================] - 5s 23ms/step - loss: 1.7239e-08 - accuracy: 1.0000 - val_loss: 0.0788 - val_accuracy: 0.9838
Epoch 2/10
208/208 [==============================] - 4s 21ms/step - loss: 1.8046e-08 - accuracy: 1.0000 - val_loss: 0.0786 - val_accuracy: 0.9838
Epoch 3/10
208/208 [==============================] - 4s 20ms/step - loss: 1.6216e-08 - accuracy: 1.0000 - val_loss: 0.0789 - val_accuracy: 0.9838
Epoch 4/10
208/208 [==============================] - 4s 21ms/step - loss: 1.4117e-08 - accuracy: 1.0000 - val_loss: 0.0789 - val_accuracy: 0.9845
Epoch 5/10
208/208 [==============================] - 4s 21ms/step - loss: 1.5678e-08 - accuracy: 1.0000 - val_loss: 0.0782 - val_accuracy: 0.9841
Epoch 6/10
208/208 [==============================] - 4s 21ms/step - loss: 1.2449e-08 - accuracy: 1.0000 - val_loss: 0.0778 - val_accuracy: 0.9843
Epoch 7/10
208/208 [==============================] - 4s 21ms/step - loss: 1.2279e-08 - accuracy: 1.0000 - val_loss: 0

In [ ]:
pred = recognizeMultiClass(model_2, xTestBoW_, 'Bow+Softmax')

НЕЙРОНКА:  Bow+Softmax

Класс: Генри      100% сеть отнесла к классу Генри     Это ПРАВИЛЬНЫЙ ответ!
Класс: Стругацкие 96% сеть отнесла к классу Стругацкие Это ПРАВИЛЬНЫЙ ответ!
Класс: Булгаков   99% сеть отнесла к классу Булгаков   Это ПРАВИЛЬНЫЙ ответ!
Класс: Клиффорд   100% сеть отнесла к классу Клиффорд  Это ПРАВИЛЬНЫЙ ответ!
Класс: Макс       97% сеть отнесла к классу Макс       Это ПРАВИЛЬНЫЙ ответ!
Класс: Брэдберри  100% сеть отнесла к классу Брэдберри Это ПРАВИЛЬНЫЙ ответ!

Средний процент распознавания 98%

